---
title: "Some tests with gediDB"
execute: 
  enabled: true
---

In [1]:
import geopandas as gpd
import gedidb as gdb
import plotly.express as px
import plotly.io as pio
from shapely.geometry import Polygon

pio.renderers.default = "iframe"

# Instantiate the GEDIProvider
provider = gdb.GEDIProvider(
    storage_type='s3',
    s3_bucket="dog.gedidb.gedi-l2-l4-v002",
    url="https://s3.gfz-potsdam.de"
)

# Load region of interest (ROI)
region_of_interest = gpd.read_file('MougueirasFire.geojson')

# Define variables to query and quality filters
vars_selected = ["agbd", 'wsci']

# Query data
gedi_data_pre = provider.get_data(
    variables=vars_selected,
    query_type="bounding_box",
    geometry=region_of_interest,
    start_time="2019-09-26",
    end_time="2020-09-27",
    return_type='dataframe'
)

gedi_data_post = provider.get_data(
    variables=vars_selected,
    query_type="bounding_box",
    geometry=region_of_interest,
    start_time="2020-09-27",
    end_time="2021-09-27",
    return_type='dataframe'
)

In [2]:
#| echo: false

gdf = gpd.GeoDataFrame(gedi_data_pre, geometry=gpd.points_from_xy(x= gedi_data_pre['longitude'], y = gedi_data_pre['latitude']))
gdf.crs = 4326

gedi_ba = gpd.sjoin(gdf, region_of_interest, how='inner').drop(columns='index_right')

# Create a grid over the extent of your points
gedi_ba.to_crs(epsg=32629, inplace=True)  # Project to UTM

bounds = gedi_ba.total_bounds  # Get the bounds of your GeoDataFrame

grid_cells = []

resolution = 250

for x0 in range(int(bounds[0]), int(bounds[2]), resolution):  # Adjust grid size (e.g., 1000 meters)
    for y0 in range(int(bounds[1]), int(bounds[3]), resolution):
        grid_cells.append(Polygon([(x0, y0), (x0 + resolution, y0), (x0 + resolution, y0 + resolution), (x0, y0 + resolution)]))

grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=gedi_ba.crs)

# Spatial join to aggregate points in each grid cell
grid = gpd.sjoin(grid, gedi_ba, how='inner')
aggregated = grid[['geometry', 'agbd', 'wsci']].dissolve(by = grid.index, aggfunc='mean').to_crs(epsg=4326)

px.choropleth_map(aggregated, geojson = aggregated.geometry, 
                  locations=aggregated.index, color='agbd',
                  color_continuous_scale="Greens",
                  range_color=(0, 100),
                  center = {'lon' : aggregated.centroid.x.mean(), 'lat' : aggregated.centroid.y.mean()},
                  zoom = 10)

/tmp/ipykernel_13742/653032167.py:29: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [3]:
#| echo: false

gdf = gpd.GeoDataFrame(gedi_data_post, geometry=gpd.points_from_xy(x= gedi_data_post['longitude'], y = gedi_data_post['latitude']))
gdf.crs = 4326

gedi_ba = gpd.sjoin(gdf, region_of_interest, how='inner').drop(columns='index_right')

# Create a grid over the extent of your points
gedi_ba.to_crs(epsg=32629, inplace=True)  # Project to UTM

bounds = gedi_ba.total_bounds  # Get the bounds of your GeoDataFrame

grid_cells = []

resolution = 250

for x0 in range(int(bounds[0]), int(bounds[2]), resolution):  # Adjust grid size (e.g., 1000 meters)
    for y0 in range(int(bounds[1]), int(bounds[3]), resolution):
        grid_cells.append(Polygon([(x0, y0), (x0 + resolution, y0), (x0 + resolution, y0 + resolution), (x0, y0 + resolution)]))

grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=gedi_ba.crs)

# Spatial join to aggregate points in each grid cell
grid = gpd.sjoin(grid, gedi_ba, how='inner')
aggregated = grid[['geometry', 'agbd', 'wsci']].dissolve(by = grid.index, aggfunc='mean').to_crs(epsg=4326)

px.choropleth_map(aggregated, geojson = aggregated.geometry, 
                  locations=aggregated.index, color='agbd',
                  color_continuous_scale="Greens",
                  range_color=(0, 100),
                  center = {'lon' : aggregated.centroid.x.mean(), 'lat' : aggregated.centroid.y.mean()},
                  zoom = 10)

/tmp/ipykernel_13742/2665748922.py:29: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


